Исходник датасета: https://www.kaggle.com/aerodinamicc/ecommerce-website-funnel-analysis


In [1]:
# Загружаем файлы c датасетами в Google Colab

In [2]:
# Импортируем нужные бибилиотеки
import pandas as pd

In [4]:
# Загружаем в Python с помощью Pandas все датасеты
user_table = pd.read_csv('data/Less_3_Data_set/user_table.csv')

# Просматриваем первые 5 строк датасета
user_table.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data/Less_3_Data_set/user_table.csv'

In [ ]:
# Просматриваем кол-во строк и столбцов каждого датасета
user_table.shape

In [ ]:
home_page = pd.read_csv('home_page_table.csv')
home_page.head()

In [ ]:
home_page.shape

In [ ]:
sum(home_page.duplicated())

In [ ]:
search_page = pd.read_csv('search_page_table.csv')
search_page.head()

In [ ]:
search_page.shape

In [ ]:
payment_page = pd.read_csv('payment_page_table.csv')
payment_page.head()

In [ ]:
payment_page.shape

In [ ]:
payment_confirmation = pd.read_csv('payment_confirmation_table.csv')
payment_confirmation.head()

In [ ]:
payment_confirmation.shape

In [ ]:
# Меняем название одинаковой колонки в каждом датасете
home_page = home_page.rename(columns = {'page':'home_page'})
search_page = search_page.rename(columns = {'page':'search_page'})
payment_page = payment_page.rename(columns = {'page':'payment_page'})
payment_confirmation = payment_confirmation.rename(columns = {'page':'payment_confirmation'})

In [ ]:
# Склеиваем последовательно все датасеты с помощью метода merge по идентификатору user_id
df = user_table.merge(home_page, on = 'user_id', how = 'left').merge(search_page, on = 'user_id', how = 'left') \
.merge(payment_page, on = 'user_id', how = 'left').merge(payment_confirmation, on = 'user_id', how = 'left')
df.head()

In [ ]:
df.shape

In [ ]:
# Второй способ: склеиваем все датасеты сразу
from functools import reduce

data_frames = [user_table, home_page, search_page, payment_page, payment_confirmation]

df_merged = reduce(lambda  left,right: pd.merge(left,right,on='user_id', how='left'), data_frames)

df_merged.head()

In [ ]:
df_merged.shape

In [ ]:
# Для удоства последующих расчетов, перекодируем названия событий в 0/1 с помощью лямбда-функции
df['home_page'] = df['home_page'].apply(lambda x: 1 if x == 'home_page' else 0)
df['search_page'] = df['search_page'].apply(lambda x: 1 if x == 'search_page' else 0)
df['payment_page'] = df['payment_page'].apply(lambda x: 1 if x == 'payment_page' else 0)
df['payment_confirmation'] = df['payment_confirmation'].apply(lambda x: 1 if x == 'payment_confirmation_page' else 0)

df.head()

In [ ]:
# Агрегируем данные: подсчитываем количество пользователей по разным типам устройств
df['device'].value_counts()

In [ ]:
# Агрегируем данные: подсчитываем общее количество пользователей и сколько из них подтвердили покупку
df['payment_confirmation'].agg({'users':'count', 'payers':'sum'})

In [ ]:
щ# Агрегируем данные: подсчитываем сколько пользователей совершило каждое из 4х событий на сайте
summary = df[['home_page', 'search_page', 'payment_page', 'payment_confirmation']].sum().reset_index()
summary.head()

In [ ]:
# Для удобства переименуем столбцы
summary= summary.rename(columns = {'index':'stage', 0:'number'})
summary.head()

In [ ]:
# Импортируем графическую библиотеку Plotly (https://plotly.com/python/funnel-charts/)
from plotly import graph_objects as go

# Настраиваем скрипт этой библиотеки для построения воронки конверсий нашими параметрами
fig = go.Figure(go.Funnel(
    y = summary['stage'],
    x = summary['number'],
    textinfo = "value+percent previous"))

fig.update_layout(
    title = 'Воронка конверсии Интернет-магазина',
    title_x=0.5,
    autosize=False,
    width=1200,
    height=600)

fig.show()

In [ ]:
# Из квиза на уроке мы помним, что лучше сегментировать воронку конверсий в разрезе характеристик пользователей
# Подготовим группировку данных по совершению событий на сайте в разрезе типов устройств, используя метод groupby
# Метод reset_index() используется для того, чтобы превратить индекс, по которому группируем данные (device), в отдельную колонку датафрейма
dfg = df.groupby('device')[['home_page', 'search_page', 'payment_page', 'payment_confirmation']].sum().unstack('device').unstack('device').reset_index()

# Уберем ненужный заголовок верхнего уровня
dfg.columns.name = None

# Переименуем стандартные после процедуры заголовки
dfg = dfg.rename(columns = {'index':'Page'})

# Посмотрим, что получилось
dfg.head()

In [ ]:
# Строим более продвинутый вариант воронки конверсий на сайте с сегментацией по типам устройств


# Настраиваем скрипт библиотеки Plotly нашими параметрами
fig = go.Figure()

# Часть графика для типа устройств Dekstop
fig.add_trace(go.Funnel(
    y = dfg['Page'],
    x = dfg['Desktop'],
    name = 'Desktop',
    constraintext='outside',
    textposition = 'inside',
    textinfo = 'value+percent previous'
    ))

# Часть графика для типа устройств Mobile
fig.add_trace(go.Funnel(
    y = dfg['Page'],
    x = dfg['Mobile'],
    name = 'Mobile',
    textposition = 'auto',
    textinfo = 'value+percent previous'
   ))

fig.update_layout(
    title = 'Воронка конверсии по типам устройств',
    title_x=0.5,
    autosize=False,
    width=1200,
    height=600)

fig.show()

А теперь потренируемся в создании сводных таблиц методами groupby, pivot_table, crosstab для ответов на поставленные вопросы:

1) Аудитория какого пола и на каких типах устройств больше всего у интернет-магазина?

In [ ]:
# Используем метод pivot_table
pd.pivot_table(df, index = 'sex', columns = 'device', values = 'home_page', aggfunc = 'sum', margins=True)

In [ ]:
# Используем функцию crosstab
# (по умолчанию рассчитывает частоты, поэтому valus = df['user_id'] и aggfunc = 'count' можно не указывать)
pd.crosstab(index = df['sex'], columns = df['device'], margins = True)

In [ ]:
# Используем метод groupby
df.groupby(['sex', 'device'])['user_id'].count()

In [ ]:
# Немного не то представление получилось, да? Попробуем добавить метод unstack()
df.groupby(['sex', 'device'])['user_id'].count().unstack('device')

2) Кто чаще всего останавливается только на поиске товаров в интернет-магазине - мужчины или женщины?

In [ ]:
# С этой задачей лучше справится groupby
(df.groupby('sex')['search_page'].sum() - df.groupby('sex')['payment_page'].sum()) / df.groupby('sex')['search_page'].sum()

3) Кто чаще подтверждает платеж и совершает окончательную покупку?

In [ ]:
# Используем функцию crosstab и ее параметр normalize для получения относительных частот 
# (по умолчанию рассчитывает частоты, поэтому valus = df['user_id'] и aggfunc = 'count' можно не указывать)
pd.crosstab(index = df['sex'], columns = df['payment_confirmation'], normalize='index')

In [ ]:
# Используем метод pivot_table (по умолчанию рассчитывает средние значения, поэтому  aggfunc = 'mean' можно не указывать)
df.pivot_table(index = ['sex'], values = 'payment_confirmation')

In [ ]:
# Используем метод groupby
df.groupby('sex')['payment_confirmation'].mean()

4) И на каких устройствах?

In [ ]:
# Используем функцию crosstab и ее параметр normalize для получения относительных частот 
pd.crosstab(index = [df['sex'], df['device']], columns = df['payment_confirmation'], normalize='index')

In [ ]:
# Используем метод pivot_table
df.pivot_table(index = ['sex','device'], values = 'payment_confirmation')

In [ ]:
# Используем метод groupby
df.groupby(['sex', 'device'])['payment_confirmation'].mean()